In [1]:
# Installs geemap package
import subprocess

try:
    import geemap
except ImportError:
    print('Installing geemap ...')
    subprocess.check_call(["python", '-m', 'pip', 'install', 'geemap'])

In [2]:
import ee
import geemap

In [3]:
Map = geemap.Map(center=[40,-100], zoom=4)
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [4]:
# Add Earth Engine dataset
# Input imagery is a cloud-free Landsat 8 composite.
l8 = ee.ImageCollection('LANDSAT/LC08/C01/T1')

image = ee.Algorithms.Landsat.simpleComposite(**{
  'collection': l8.filterDate('2018-01-01', '2018-12-31'),
  'asFloat': True
})

# Use these bands for prediction.
bands = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B10', 'B11']

# Load training points. The numeric property 'class' stores known labels.
points = ee.FeatureCollection('users/kkgwasiwa/NewFc_Bokaa_2019wt_Tx')

# This property of the table stores the land cover labels.
label = 'lc'

# Overlay the points on the imagery to get training.
training = image.select(bands).sampleRegions(**{
  'collection': points,
  'properties': [label],
  'scale': 30
})

# Train a CART classifier with default parameters.
trained = ee.Classifier.smileRandomForest(10).train(training, label, bands)

# Classify the image with the same bands used for training.
classified = image.select(bands).classify(trained)

# Load the land mask from the SRTM DEM.
landMask = ee.Image('projects/ee-kkgwasiwa/assets/c2019Bokaa_Variance4_Clip1').mask()

# Update the NDVI difference mask with the land mask.
maskedDifference = classified.updateMask(landMask)

# Display the inputs and the results.
Map.centerObject(points, 11)
Map.addLayer(image, {'bands': ['B4', 'B3', 'B2'], 'max': 0.4}, 'image')
Map.addLayer(maskedDifference,
             {'min': 0, 'max': 2, 'palette': ['red', 'green', 'blue']},
             'classification')



In [5]:
Map.addLayerControl() # This line is not needed for ipyleaflet-based Map.
Map

Map(center=[-24.58121682010448, 25.601123742313437], controls=(WidgetControl(options=['position', 'transparent…